In [142]:
import requests
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import tiktoken
import torch
from transformers import AutoModelForCausalLM

In [48]:
def iterate_examples():
    with open('benchmarks/hellaswag/hellaswag_val.jsonl', 'rb') as file:
        for line in file:
            example = json.loads(line)
            yield example

In [49]:
examples = iterate_examples()

In [163]:
def prepare_example(example):
    ctx = example['ctx']
    endings = example['endings']
    label = example['label']
    
    enc = tiktoken.get_encoding('gpt2')
    ctx_tokens = enc.encode(ctx)
    tok_rows = []
    mask_rows = []
    for end in endings:
        end_tokens = enc.encode(' '+ end)
        tok_rows.append(ctx_tokens + end_tokens)
        mask_rows.append([0] * len(ctx_tokens) + [1] * len(end_tokens))

    tokens = torch.zeros([4,1024], dtype=torch.long)
    mask = torch.zeros([4,1024], dtype=torch.long)
    for i,(tok_row, mask_row) in enumerate(zip(tok_rows, mask_rows)):
        tokens[i,:len(tok_row)] = torch.tensor([tok_row]) 
        mask[i,:len(mask_row)] = torch.tensor([mask_row])
    
    
    return tokens, mask, label

In [157]:
tokens = torch.zeros([4,1024], dtype=torch.long)

In [160]:
tokens[0, :10] = torch.tensor([1] * 10)

In [161]:
tokens

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [164]:
prepare_example(ex)

(tensor([[7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 2)

In [147]:
ten[0,-11:]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
@torch.no_grad()
def evaluate():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
    model.to(device)
    hella_val = iterate_examples()
    for example in hella_val:
        tokens, mask, lable = prepare_example(example)
        tokens, mask = tokens.to(device), mask.to(device)
        model(tokens) 

(tensor([[7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0],
         [7571, 1466,  287,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 2)

In [79]:
label

2

In [82]:
full_sentenses = list(map(lambda x: ctx + ' ' +  x, endings))

In [128]:
torch.tensor(tokenized)

tensor([[7571, 1466,  287,  ...,    0,    0,    0],
        [7571, 1466,  287,  ...,    0,    0,    0],
        [7571, 1466,  287,  ...,    0,    0,    0],
        [7571, 1466,  287,  ...,    0,    0,    0]])

In [126]:
len((mask[0]))

1024

In [121]:
lengths

[246, 246, 216, 278]